In [ ]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

### American revolution 

<img src="images/13_colonies.jpg" width="400px" align="right"/>
The American Revolution (1775-83) is also known as the American Revolutionary War and the U.S. War of Independence.

Independance means not beeing under the control of others and beeing able to make one's own decisions. On July 4, 1776,  the Thirteen Colonies declared that they were no longer part of Britain. The British disagreed. A war between Britain and the Thirteen Coloniws began as a result of this disagreement. This war was known as the American Revolution or the War of Independence. (From "Our Land and People", Patricia Shields-Ramsay, Douglas Ramsay)



#### Displaying battles on the map

*Step1*. Loading python libraries.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import time
init_notebook_mode(connected=True)

*Step2*. Downloading dataset  and examining it.

In [ ]:
url = 'https://data.opendatasoft.com/explore/dataset/list-of-american-revolutionary-war-battles@public-us/download/?format=csv&timezone=America/Denver&use_labels_for_header=true'
revolution_data  = pd.read_csv(url,sep=';',error_bad_lines=False)
revolution_data.head()

*Step3*. To display battles on the map we neeed longitude and latitude - splitting "Battle location" column into two: "lon" and "lat".
To use a timeline slider we neeed a year  - createing  "Year" column based on first 4 characters of the "Start Date" column.
In adition cleaning some empty values from the dataset(based on  "Battle location" and "Start Date" columns).
Examining dataset again.

In [ ]:
revolution_data = revolution_data.dropna(subset=['Battle location'])
revolution_data = revolution_data.dropna(subset=['Start Date'])
revolution_data['lat'] = revolution_data['Battle location'].str.split(', ', expand = True)[0]
revolution_data['lon'] = revolution_data['Battle location'].str.split(', ', expand = True)[1]
revolution_data['Year'] = revolution_data['Start Date'].str[:4]
revolution_data=revolution_data[revolution_data['Year'] != '2026']
#empty row to make legend working
revolution_data.loc[revolution_data.shape[0]]=[' ',' ',' ',' ',' ',' ','Patriot victory',' ','0','0','1783']
revolution_data.head()

*Step4*.Displaying battles on the map based on the start date and victory(North America).

In [ ]:
years_from_col = set(revolution_data['Year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]
#years.remove('2026')
#print(years)
revolution_data['Victory'] = revolution_data['Victory'].replace(np.nan, 'Unknown', regex=True)
victories = []
for victory in revolution_data['Victory']:
    if victory not in victories: 
        victories.append(victory)       
#print(victories)

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1774',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

custom_colors = {
    'Patriot victory': 'rgb(171, 0, 250)',
    'Unknown': 'rgb(150, 99, 0)',
    'British victory': 'rgb(99, 110, 250)',
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
figure['layout']['title'] = ' American Revolution (1775-83) battles'
figure['layout']['geo']  =  dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation = dict(
                lon = -100
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
)
 

year = 1774
for victory in victories:
    dataset_by_year = revolution_data[revolution_data['Year'] == year]
    dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

    data_dict = {
        'type': 'scattergeo',
        'lon': list(dataset_by_year_and_vict['lon']),
        'lat': list(dataset_by_year_and_vict['lat']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_vict['Battle']),
        'marker': {
            'size': 8, 
            'color': custom_colors[victory]
        },
        'name': victory,
        'legendgroup' : 'group1'
    }
    figure['data'].append(data_dict)
    

for year in years:
    frame = {'data': [], 'name': str(year)}
    for victory in victories:
        dataset_by_year = revolution_data[revolution_data['Year'] == year]
        dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

        data_dict = {
            'type': 'scattergeo',
            'lon': list(dataset_by_year_and_vict['lon']),
            'lat': list(dataset_by_year_and_vict['lat']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_vict['Battle']),
            'marker': {
                'size': 8, 
                'color': custom_colors[victory]
            },
            'name': victory,
            'legendgroup' : 'group'
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, validate=False)

*Step5*. How may battles per year? Creating table and coloring rows according to the battle count.

In [ ]:
import colorlover as cl
results=revolution_data.groupby(['Year']).size().reset_index(name='Count').sort_values(by=['Count'])
results= results[results['Year']!=" "].reset_index(drop=True)
results["Colour"]=['rgb(255,255,255)'] + cl.scales['9']['seq']['Blues'] 
results=results.sort_values(by=['Year'])
trace0 = go.Table(
  type = 'table',
  header = dict(
    values = [ "<b>YEAR</b>","<b>NUMBER OF BATTLES</b>",],
    line = dict(color = 'white'),
    fill = dict(color = 'white'),
    align = ['center'],
    font = dict(color = 'black', size = 12)
  ),
  cells = dict(
    values = [results.Year, results.Count],
    line = dict(color = [results.Colour]),
    fill = dict(color = [results.Colour]),
    align = 'center',
    height = 30,
    font = dict(color = 'green', size = 20)
    ))

data = [trace0]

iplot(data)

*Step7*.Number of battles by location  - calculating percentage of battles by locataion.

In [ ]:
results=revolution_data.groupby(['State']).size().reset_index(name='Count').sort_values(by=['Count'],ascending=False)
results= results[results['State']!=" "].reset_index(drop=True)
results['Percent'] = round(results['Count']*100/sum(results['Count']),2).astype(str) +"%"
trace0 = go.Bar(
    x=results['State'],
    y=results['Count'],
    text=results['Percent'],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Number of battles by location',
    font=dict(size=18),
    margin = dict(
    l= 30,
    r= 30,
    t= 50,
    b= 250
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

*Step7*. One of the battles short description.
#### Naval battle off Halifax
<img src="images/Halifax_battle.jpg" width="400px" align="right"/>
The Battle off Halifax took place on 28 May 1782 during the American Revolutionary War. It involved the American privateer Jack and the 14-gun Royal Naval brig HMS Observer off Halifax, Nova Scotia. Captain David Ropes commanded Jack, and Lieutenant John Crymes commanded Observer. The battle was "a long and severe engagement."

Observer was returning to Halifax, having rescued from American privateer Noah Stoddard ten crew members of HMS Blonde after Blonde wrecked on Seal Island off Cape Sable Island.[9] On 28 May 1782, as Observer arrived at the Sambro Island Light near the mouth of Halifax Harbour, Jack approached her. When Jack discovered her quarry was a British naval vessel, the Americans tried to escape.

Observer chased Jack for two hours before catching her. The ships were evenly matched. The British immediately killed the American captain David Ropes as a result of the cannon fire. Both ships had numerous holes shot through their sails, and the British sailors attempted to climb the rigging of their ship in an effort to board the American privateer. The Americans repulsed this initial boarding attempt, but the British were ultimately successful. Jack struck her colours on the afternoon of 29 May.